### Imports and definitions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import pickle, sys, os, json
import astropy.units as u
import pandas as pd
pd.set_option("display.max_columns", None)

from astropy.coordinates import SkyCoord
from matplotlib.dates    import DayLocator, MonthLocator, DateFormatter
from regions             import PointSkyRegion
from astropy.time        import Time
from scipy.stats         import chi2
from IPython.display     import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from gammapy.modeling.models import create_crab_spectral_model, SkyModel, LogParabolaSpectralModel
from gammapy.estimators      import FluxPointsEstimator, LightCurveEstimator, FluxPoints
from gammapy.modeling        import Fit
from gammapy.datasets        import Datasets, SpectrumDataset
from gammapy.makers          import SpectrumDatasetMaker, WobbleRegionsFinder, ReflectedRegionsBackgroundMaker, SafeMaskMaker
from gammapy.maps            import MapAxis, RegionGeom, Map, TimeMapAxis
from gammapy.data            import DataStore

# import scripts
sys.path.insert(0, os.path.join("/fefs/aswg/workspace/juan.jimenez/lst1_systematics/scripts"))
import auxiliar  as aux
import documents as docs

# ============================ #
# dl3 path where dl3 and index files are
# dl3_dir = "/fefs/aswg/workspace/daniel.morcuende/data/real/DL3/Crab_performance/AllSkyMC_v0.9.9/intensity80/all_nodes/gh_eff_0.7_th_cont_0.7"
dl3_dir = "/fefs/aswg/workspace/juan.jimenez/data/lst1_systematics/dl3"

fname_dict = "objects/dict_sed_and_lc.pkl"
# ============================ #

### Loading configuration files

In [ ]:
# reading the configuration from the gammapy configuration file
target_name, n_off_regions, _e_reco, _e_true = docs.load_gammapy_analysis_configuration()

e_reco_min, e_reco_max, e_reco_bin_p_dec = _e_reco["min"], _e_reco["max"], _e_reco["bins_p_dec"]
e_true_min, e_true_max, e_true_bin_p_dec = _e_true["min"], _e_true["max"], _e_true["bins_p_dec"]


### Loading full datasets

In [ ]:
# Opening all the dl3 data in a path
total_data_store = DataStore.from_dir(dl3_dir)

# Taking obs ids
obs_ids = total_data_store.obs_table["OBS_ID"].data
obs_ids = obs_ids[:]

# Then we get the observation information from the total data store
observations = total_data_store.get_observations(
    obs_ids,
    required_irf=["aeff", "edisp", "rad_max"]
)

# Defining target position and ON reion
target_position = SkyCoord.from_name(target_name, frame='icrs')
on_region = PointSkyRegion(target_position)

print(f'Total livetime of observations {total_data_store.obs_table["LIVETIME"].data.sum()/3600:.2f} h')
display(total_data_store.obs_table[:5])

### Defining all the energy axes

In [ ]:
# ============================ #
# estimated energy axes
energy_axis = MapAxis.from_energy_bounds(
    e_reco_min, e_reco_max, 
    nbin=e_reco_bin_p_dec, per_decade=True, 
    unit="TeV", name="energy"
)
# ============================ #
# estimated energy axes
energy_axis_true = MapAxis.from_energy_bounds(
    e_true_min, e_true_max, 
    nbin=e_true_bin_p_dec, per_decade=True, 
    unit="TeV", name="energy_true"
)
# ============================ #
# Energy for the spectrum
e_fit_min = energy_axis.edges[1].value
e_fit_max = energy_axis.edges[-1].value
e_fit_bin_p_dec = e_reco_bin_p_dec

# Just to have a separate MapAxis for spectral fit energy range
energy_fit_edges = MapAxis.from_energy_bounds(
    e_fit_min, e_fit_max, 
    nbin=e_fit_bin_p_dec, per_decade=True, 
    unit="TeV"
).edges

# ============================ #
# Energy for the lightcurve
e_lc_min = energy_axis.edges[0]
e_lc_max = energy_axis.edges[-1]

print("Spectral fit will be done in energy edges:\n", energy_fit_edges)
print(f"\nLC will be estimated from {e_lc_min:.1f} to {e_lc_max:.1f}")

### Getting time parameters

In [ ]:
# # Get the GTI parameters of each observations to create time intervals for plotting LC
# t_start  = []
# t_stop   = []
# tot_time = []

# for obs in observations:
#     gti = obs.gti
    
#     t_start.append( gti.time_start[0])
#     t_stop.append(  gti.time_stop[0])
#     tot_time.append(gti.time_sum.value)

# t_start  = np.sort(np.array(t_start))
# t_stop   = np.sort(np.array(t_stop))
# tot_time = np.array(tot_time)

# t_start = Time(t_start)
# t_stop  = Time(t_stop)

# t_day = np.unique(np.rint(t_start.mjd))

# # To make the range night-wise, keep the MJD range in half integral values
# t_range = [Time([t-0.5, t+0.5], format="mjd", scale="utc") for t in t_day]

### We define the geometry regions in te sky and prepare the empty datasets and makers

In [ ]:
# geometry defining the ON region and SpectrumDataset based on it
geom = RegionGeom.create(
    region=on_region, 
    axes=[energy_axis]
)

# creating an empty dataset
dataset_empty = SpectrumDataset.create(
    geom=geom, 
    energy_axis_true=energy_axis_true
)
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False,
    selection=["counts", "exposure", "edisp"]
)

# tell the background maker to use the WobbleRegionsFinder
region_finder = WobbleRegionsFinder(n_off_regions=n_off_regions)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

### Now we analize the ON and OFF regions in the dataset and we store them in `datasets`, then the datasets can be stacked in a unique one

In [ ]:
%%time
# The final object will be stored as a Datasets object
datasets = Datasets()

for observation in observations:
    dataset = dataset_maker.run(
        dataset=dataset_empty.copy(name=str(observation.obs_id)),
        observation=observation
    )
    dataset_on_off = bkg_maker.run(
        dataset=dataset, 
        observation=observation
    )
    datasets.append(dataset_on_off) 

# Stacking all the datasets in one
stacked_dataset = Datasets(datasets).stack_reduce()
print(stacked_dataset)

### Then we define the model and set inside the dataset

In [ ]:
# defining the model we want to fit and the starting values
spectral_model = LogParabolaSpectralModel(
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    alpha=2,
    beta=0.1,
    reference=1 * u.TeV,
)
# we will use the crab model in general
model = SkyModel(
    spectral_model=spectral_model, 
    name="crab"
)

# We set the model of all datasets to log parabola
stacked_dataset.models = model

### We fit the model with the stacked dataset

In [ ]:
%%time
# Now we run the fit to extract the parameters of the model
fit = Fit()
result = fit.run(datasets=stacked_dataset)
best_fit_model = model.copy()

display(stacked_dataset.models.to_parameters_table())

### Then from the model and the data we can extract the flux points

In [ ]:
%%time 
# then extracting the flux points from the data
fpe = FluxPointsEstimator(
    energy_edges=energy_fit_edges, 
    source=target_name, 
    selection_optional="all"
)

# We apply the flux point estiation from the datasets
flux_points = fpe.run(datasets=stacked_dataset)
flux_points.to_table(sed_type="dnde", formatted=True)[:5]

### Then we can plot the SED

In [ ]:
plot_kwargs = {
    "energy_bounds": [0.08, 100] * u.TeV,
    "sed_type": "e2dnde",
    "yunits": u.Unit("TeV cm-2 s-1"),
    "xunits": u.TeV,
}

# Crab models
crab_magic_100 = create_crab_spectral_model("magic_lp")
crab_magic_10  = create_crab_spectral_model("magic_lp")
crab_magic_1   = create_crab_spectral_model("magic_lp")
crab_magic_10.amplitude.value = crab_magic_10.amplitude.value * 0.1
crab_magic_1.amplitude.value  = crab_magic_1.amplitude.value  * 0.01

fig, ax = plt.subplots(figsize=(10,5))


best_fit_model.spectral_model.plot(
    ax=ax, ls="-", lw=1.5, color="k", label="best fit", **plot_kwargs
)

best_fit_model.spectral_model.plot_error(
    ax=ax, facecolor="k", alpha=0.2, **plot_kwargs
)

crab_magic_100.plot(
    ax=ax, ls="--", lw=1.5, color="crimson", label="MAGIC reference (Aleksić et al. 2015)", **plot_kwargs
)
# crab_magic_10.plot(
#     ax=ax, ls="--", lw=1.5, color="crimson", label="Crab 10%", **plot_kwargs
# )
# crab_magic_1.plot(
#     ax=ax, ls="--", lw=1.5, color="crimson", label="Crab 1%", **plot_kwargs
# )

flux_points.plot(sed_type="e2dnde", color="k", label="Flux points")
flux_points.plot_ts_profiles(sed_type="e2dnde", cmap=cmap)

ax.legend(loc=3)
ax.set_ylim([1e-13, 1e-10])
ax.grid(which="both", alpha=0.5)
plt.show()

### Then once we have found the SED model we fix the alpha and beta parameters and let the amplitude as a free parameter

In [ ]:
model.parameters["alpha"].frozen = True
model.parameters["beta"].frozen  = True

# Create the LC Estimator for each run
lc_maker_1d = LightCurveEstimator(
    energy_edges=[e_lc_min, e_lc_max], 
    reoptimize=False, # Re-optimizing other free model parameters (not belonging to the source)
    source="crab", 
    selection_optional="all" # Estimates asymmetric errors, upper limits and fit statistic profiles
)

# Assigning the fixed parameters model to each dataset
for data in datasets:
    data.models = model

### Then we run the lightcurve maker run-wise

In [ ]:
%%time
print(f"LC will be estimated from {e_lc_min:.1f} to {e_lc_max:.1f}")

lc_runwise = lc_maker_1d.run(datasets)
lightcurve = lc_runwise.to_table(sed_type="flux", format="lightcurve")

### We calculate the mean flux and the statistical error

In [ ]:
def weighted_average(table, sys_error=0):
    val = table["flux"]
    uncertainty = np.sqrt((sys_error * table["flux"])**2 + table["flux_err"]**2)
    return (val/uncertainty**2).sum() / (1/uncertainty**2).sum(), np.sqrt(1/np.sum(1/uncertainty**2))


mean_flux, mean_flux_err = weighted_average(lightcurve)

### Calculating the $\chi^2$ and $p$ values

In [ ]:
def calculate_chi2_pvalue(table, sys_error=0):
    uncertainty = np.sqrt((sys_error * table["flux"])**2 + table["flux_err"]**2)
    flux = table["flux"]
    mean_flux = (flux/uncertainty**2).sum() / (1/uncertainty**2).sum()
    mean_flux_err = np.sqrt(1/np.sum(1/uncertainty**2))
    print(f"Weighted mean flux: {mean_flux:.3e} +/- {mean_flux_err:.3e} cm-2 s-1")
    
    chi2_value = np.sum((table["flux"] - mean_flux)**2/uncertainty**2)
    ndf = len(table["flux"]) - 1
    pvalue = chi2.sf(x=chi2_value, df=ndf)
    print(f"Chi2: {chi2_value:.1f}, ndf: {ndf}, P-value: {pvalue:.2e}")
    return chi2_value, ndf, pvalue
    
calculate_chi2_pvalue(lightcurve, sys_error=0.0);

### Extracting the data from the table as arrays

In [ ]:
# Start time, duration and central time
time_min = Time(np.hstack(lightcurve["time_min"]), format='mjd').datetime
time_max = Time(np.hstack(lightcurve["time_max"]), format='mjd').datetime
delta_time  = time_max - time_min
time_center = time_min + delta_time / 2

# Flux and flux error
flux_lst1 = np.hstack(lightcurve["flux"])
flux_stat_err_lst1 = np.hstack(lightcurve["flux_err"])

# run numbers
run_num = [int(n) for n in observations.ids]

### The Crab Nebula reference from MAGIC

In [ ]:
crab = create_crab_spectral_model("magic_lp")

crab.amplitude.error = 0.03e-11 * u.Unit("cm-2 s-1 TeV-1")
crab.alpha.error = 0.01
crab.beta.error = 0.01/np.log(10)


flux_crab = crab.integral(e_lc_min, e_lc_max)
flux_crab_error = flux_crab * 0

### Plotting the LC

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

# Plotting the Ligt Curve
ax.errorbar(time_center, flux_lst1, yerr=flux_stat_err_lst1, color="k", ls="", marker=".", ms=10, label="LST-1 (src-independent)")
    
# Mean flux + error
ax.axhline(mean_flux, ls="--", color="k", zorder=-1, label="Mean flux")
ax.axhspan(mean_flux - mean_flux_err, mean_flux + mean_flux_err, color="k", alpha=0.4, zorder=-1)

# MAGIC reference
ax.axhline(flux_crab.value, ls="-.", color="crimson", zorder=-1, label="MAGIC (Aleksić et al. 2015)")

energy_range = f"{e_lc_min:.1f} < $E$ < {e_lc_max:.1f}"
ax.set_title(f"Light curve of {target_name} ({energy_range})")
ax.set_ylabel("Flux [cm$^{-2}$ s$^{-1}$]")
ax.set_xlabel("Time [date]")
ax.grid()
ax.legend(loc=4)
ax.xaxis.set_major_locator(MonthLocator(interval=1))
ax.xaxis.set_major_formatter(DateFormatter("%m-%Y"))
plt.xticks(rotation=45)


plt.show()

### Plotting the LC run-wise

In [ ]:
fig, ax = plt.subplots(figsize=(17,3))

# Plotting the Ligt Curve
ax.errorbar(np.arange(len(flux_lst1)), flux_lst1, yerr=flux_stat_err_lst1, color="k", ls="", marker=".", ms=10, label="LST-1 (src-independent)")  

# Mean flux + error
ax.axhline(mean_flux, ls="--", color="k", zorder=-1, label="Mean flux")
ax.axhspan(mean_flux - mean_flux_err, mean_flux + mean_flux_err, color="k", alpha=0.4, zorder=-1)

# MAGIC reference
ax.axhline(flux_crab.value, ls="-.", color="crimson", zorder=-1, label="MAGIC (Aleksić et al. 2015)")

ax.legend()
ax.grid()
ax.set_xlabel(f"Run #")
ax.set_ylabel("Flux [cm$^{-2}$ s$^{-1}$]")
energy_range = f"{e_lc_min:.1f} < $E$ < {e_lc_max:.1f}"
ax.set_title(f"Light curve of {target_name} ({energy_range})")
plt.show()

### Now we put all inside a dict and we store it in a file

In [ ]:
dict_total = {
    
    "dict_model" : best_fit_model.to_dict(), # SkyModel.from_dict(<>)
    
    "table_sed"  : flux_points.to_table(),   # FluxPoints.from_table(<>)

    "lightcurve" : {

        "run_number" : run_num,
        "t_start"    : time_min,
        "t_stop"     : time_max,
        "timedelta"  : delta_time,
        "flux"       : flux_lst1,
        "e_flux"     : flux_stat_err_lst1,
        
        "global" : {
            "e_min" : e_lc_min,
            "e_max" : e_lc_max,
            "n_off_regions" : n_off_regions,
            "target_name"   : target_name
        }
    }
}


# Saving the object
with open(fname_dict, 'wb') as f:
    pickle.dump(dict_total, f, pickle.HIGHEST_PROTOCOL)